# Data Import and Preprocessing

### Install and import necessary packages

In [3]:
# install record linkage package
%pip install recordlinkage

     |████████████████████████████████| 952kB 1.2MB/s eta 0:00:01
     |████████████████████████████████| 143kB 9.7MB/s eta 0:00:01
  Created wheel for jellyfish: filename=jellyfish-0.7.2-cp37-cp37m-linux_x86_64.whl size=87395 sha256=32f5962ca173686fb7a339a591d6e6d4af4da67a37d258674dd5056750cd8d3e
  Stored in directory: /home/jovyan/.cache/pip/wheels/e8/fe/99/d8fa8f2ef7b82a625b0b77a84d319b0b50693659823c4effb4
Successfully built jellyfish
Note: you may need to restart the kernel to use updated packages.


In [4]:
# general use imports
import pandas as pd
import numpy as np
import os
import glob

# Machine learing
import sklearn

# record linkage 
import recordlinkage as rl
from recordlinkage.preprocessing import clean, phonenumbers, phonetic

print( "Imports loaded at " + str( pd.datetime.now() ) )

Imports loaded at 2020-03-26 22:37:26.426103


### Load Patents Data

In [5]:
# change directory to load patents data
data_dir_p = "/home/jovyan/Yandex.Disk/BigDataPubPol/data/patents"
os.chdir(data_dir_p)

In [ ]:
# generate an empty dataframe that will hold all the patent data we have
patents = pd.DataFrame([])

# read in the files
for counter, file in enumerate(glob.glob("patent*?")):
    print(counter,file)
    patent = pd.read_csv(file)
    patents = patents.append(patent)

In [ ]:
# check
patents.head()

compare.exact() is the deterministic record linkage function provided by the recordlinkage package.

### Load Grants Data

In [6]:
# change directory to load data
data_dir_g = "/home/jovyan/Yandex.Disk/BigDataPubPol/data/projects"
os.chdir(data_dir_g)

In [9]:
# generate an empty dataframe that will hold all the patent data we have
grants = pd.DataFrame([])

# read in the files
for counter, file in enumerate(glob.glob("FedRePORTER_PRJ_C_FY*?")):
    print(counter,file)
    grant = pd.read_csv(file, low_memory=False, skipinitialspace=True)
    grants = grants.append(grant)

0 FedRePORTER_PRJ_C_FY2014.csv
1 FedRePORTER_PRJ_C_FY2015.csv
2 FedRePORTER_PRJ_C_FY2011.csv
3 FedRePORTER_PRJ_C_FY2010.csv
4 FedRePORTER_PRJ_C_FY2016.csv
5 FedRePORTER_PRJ_C_FY2017.csv
6 FedRePORTER_PRJ_C_FY2013.csv
7 FedRePORTER_PRJ_C_FY2018.csv
8 FedRePORTER_PRJ_C_FY2012.csv


In [10]:
# check
grants.head()

,PROJECT_ID,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,CONTACT_PI_PROJECT_LEADER,...,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS
0,589214,Bees; Chemicals; Development; Devices; Diet; ...,IMPROVE NUTRITION FOR HONEY BEE COLONIES TO ST...,USDA,ARS,NaN,ARS-0425868,10/1/2013,2/6/2014,"HOFFMAN, GLORIA D",...,TUCSON,AZ,85719,UNITED STATES,NaN,NaN,10.001,2014,NaN,NaN
1,589224,Affect; Algae; Amendment; Appearance; base; B...,ALGAL-BASED WATER TREATMENT TECHNOLOGIES FOR S...,USDA,ARS,NaN,ARS-0426136,1/1/2014,12/31/2018,"HALL, DAVID GOODSELL",...,FORT PIERCE,FL,34945,UNITED STATES,NaN,NaN,10.001,2014,NaN,NaN
2,591601,Area; base; Biological; California; Consumpti...,BIOLOGICAL CONTROL OF PIERCE'S DISEASE OF GRAP...,USDA,NIFA,NaN,0212205,8/20/2014,9/30/2014,"HOPKINS, DO, .",...,GAINESVILLE,FL,32611-0110,UNITED STATES,NaN,NaN,10.203,2014,NaN,NaN
3,593659,absorption; Address; Anemia; Anemia due to Ch...,MOLECULAR MECHSNISMS OF INTESTINAL METAL ION T...,USDA,NIFA,NaN,0217191,7/8/2014,7/9/2014,"COLLINS, JA, F..",...,GAINESVILLE,FL,32611-0110,UNITED STATES,NaN,NaN,10.203,2014,NaN,NaN
4,597171,Climate; Decision Support Systems; Developmen...,ASSESSING CLIMATE INFORMATION NEEDS AND OPPORT...,USDA,NIFA,NaN,0226918,8/19/2014,8/20/2014,"JONES, J.",...,GAINESVILLE,FL,32611-0110,UNITED STATES,NaN,NaN,10.202,2014,NaN,NaN


# Clean Patents Data

In [ ]:
# Check names columns
patents['patent_firstnamed_inventor_name_first'].unique().tolist()[50:100]
patents['patent_firstnamed_inventor_name_last'].unique().tolist()[50:100]

We have some names that have only a first name, for some we have a first and middle name or inital, and then there are also hyphens. So we need to create one variable fname that only contains the first name. The last name column looks better. We just need to make sure that all letters are lowercase.

In [ ]:
# remove special characters in names and make them lowercase
patents['name_last']=(clean(patents['patent_firstnamed_inventor_name_last'], 
                                 lowercase=True, remove_brackets=True))
patents['name_first']=(clean(patents['patent_firstnamed_inventor_name_first'], 
                                  lowercase=True, remove_brackets=True))

In [ ]:
# Compare the orginal names with the manipulates ones
patents[['patent_firstnamed_inventor_name_last','patent_firstnamed_inventor_name_first', 
              'name_last','name_first']].head(10)

In [ ]:
# Only keep the first name by splitting the name string
patents['name_first'] = patents.name_first.str.split(' ').str.get(0)
patents['name_middle'] = patents.name_first.str.split(' ')

In [ ]:
patents[['patent_firstnamed_inventor_name_last','patent_firstnamed_inventor_name_first', 
              'name_last','name_first']].head(10)

First and last name column is now clean.

In [ ]:
# Check state column
patents['patent_firstnamed_assignee_state'].unique()

Seems like states are recorded using abbreviation and uppercase. We also have missing values. Also seems like we have some abbreviations in here that might not be mainland US. We can count the entries to check.

In [ ]:
# Check the number of unique entries under the state column
print("We have " + str(patents['patent_firstnamed_assignee_state'].nunique()

Our data includes all US territories and DC. We can get a list of abbreviations of these states and check our entries against this list. But this looks good, we don't have to clean much here.

In [ ]:
# Fetch only the rows with valid state information
listUsStates=['AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'GU', 'HI', 'IA', 
              'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 
              'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 
              'SD', 'TN', 'TX', 'UM', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY']

patents_US = patents.loc[patents['patent_firstnamed_assignee_state'].isin(listUsStates)]

In [ ]:
# Comparing the counts before and after we can see that we lost some rows 
print(len(patents_1018['patent_firstnamed_assignee_state']))
print(len(patents_1018_US['patent_firstnamed_assignee_state']))

# Probably the nans
patents_1018_US['patent_firstnamed_assignee_state'].unique()